# Converter pgn to liquipedia format

In [ ]:
!pip install chess
!pip install ipywidgets
import codecs
from chess.pgn import read_game
import re
from io import StringIO
import ipywidgets as widgets
from ipywidgets import Accordion, FileUpload
from IPython.display import display, Javascript, Markdown

players_to_filter = []
round_to_filter = 0
win_points = 1
draw_points = 0,5
loss_points = 0.0
points_changed = False

# Upload your pgn file here

In [ ]:
pgn_file = FileUpload(accept='.pgn', multiple=False)
display(pgn_file)

# Specify players and round to filter, change point system if needed

In [ ]:
def save_filter(button):
    global players_to_filter
    global round_to_filter
    global win_points
    global draw_points
    global loss_points

    players_to_filter = text_area.value.splitlines()
    round_to_filter = round_selector.value
    win_points = win_points_selector.value
    draw_points = draw_points_selector.value
    loss_points = loss_points_selector.value

    print("Saved!")

def on_points_change(change):
    global points_changed
    points_changed = True

win_points_selector = widgets.FloatText(
    value=1,
    description='Win:',
    min=0
)

draw_points_selector = widgets.FloatText(
    value=0.5,
    description='Draw:',
    min=0
)

loss_points_selector = widgets.FloatText(
    value=0,
    description='Lose:',
    min=0
)

round_selector = widgets.IntText(
    value=0,
    description='Round:',
    min=0,
    max=1000
)

text_area = widgets.Textarea(
    value='',
    placeholder='Smyslov, Vassily\nGeller, Efim\nDueckstein, Andreas\nVassily Smyslov\nEfim Geller\nAndreas Dueckstein',
    description='Players',
    layout={'width': '80%', 'height': '200px'}
)

win_points_selector.observe(on_points_change, names='value')
draw_points_selector.observe(on_points_change, names='value')
loss_points_selector.observe(on_points_change, names='value')

save_button = widgets.Button(description="Save")
save_button.on_click(save_filter)
display(
    widgets.VBox([
        round_selector,
        widgets.HBox([win_points_selector, draw_points_selector, loss_points_selector]),
        text_area,
        save_button
    ])
)

# Keep running the code

In [ ]:
def swap_name_surname(full_name):
    parts = full_name.split(',')
    if len(parts) < 2:
        return full_name
    surname = parts[0].strip()
    name = parts[1].strip()
    return f"{name} {surname}"

def format_result(result):
    return {"1-0": 1, "0-1": 2}.get(result, 0)

def custom_score(result, white):
    if result == 1:
        return win_points if white else loss_points
    elif result == 2:
        return loss_points if white else win_points
    else:
        return draw_points

def parse_pgn():
  matches = []
  while True:
    game_info = {}
    game = read_game(pgn)
    if game is None:
      break

    game_info["white"] = game.headers.get("White", "")
    game_info["white_formated"] = swap_name_surname(re.sub(r'\b[A-Z]{3}\b', '', game_info["white"]))
    game_info["white_team"] = game.headers.get("WhiteTeam", "")
    game_info["black"] = game.headers.get("Black", "")
    game_info["black_formated"] = swap_name_surname(re.sub(r'\b[A-Z]{3}\b', '', game_info["black"]))
    game_info["black_team"] = game.headers.get("BlackTeam", "")
    game_info["date"] = game.headers.get("UTCDate", "") or game.headers.get("Date", "")
    game_info["date_formated"] = re.sub(r'-\?\?|\?{4}', '', game_info["date"].replace('.', '-'))
    game_info["time"] = game.headers.get("UTCTime", "") or game.headers.get("Time", "")
    game_info["time_formated"] = f" - {game_info['time']}" if game_info.get('time') else ''
    game_info["result"] = game.headers.get("Result", "")
    game_info["result_formated"] = format_result(game_info["result"])
    game_info["round"] = game.headers.get("Round", "").replace(",", ".").split(".")[0]
    game_info["eco"] = game.headers.get("ECO", "")
    game_info["moves"] = game.end().board().fullmove_number
    if points_changed:
      game_info["white_score"] = custom_score(game_info["result_formated"], True)
      game_info["black_score"] = custom_score(game_info["result_formated"], False)

    matches.append(game_info)
  return matches

In [ ]:
print(list(pgn_file.value[0]))

# Run this code again if you changed some settings

In [ ]:
pgn = StringIO(codecs.decode(pgn_file.value[0].content, encoding="utf-8"))
games = parse_pgn()
output_text = ""
for game_info in games:
  #filter players
  if players_to_filter and all(
    player not in players_to_filter
    for player in (
        game_info["white"],
        game_info["black"],
        game_info["white_formated"],
        game_info["black_formated"])):
    continue
  #filter round
  if (round_to_filter == 0) or (str(round_to_filter) == game_info["round"]):
    output_text += (
f'''|{{{{Match
  |finished=true
  |date={game_info["date_formated"]}{f'{game_info["time_formated"]} {{{{abbr/UTC}}}}'if game_info.get('time_formated') else ''}
  |opponent1={{{{1Opponent|flag=|{game_info["white_formated"]}{f'|score={game_info["white_score"]}'if game_info.get('white_score') else ''}}}}}
  |opponent2={{{{1Opponent|flag=|{game_info["black_formated"]}{f'|score={game_info["black_score"]}'if game_info.get('black_score') else ''}}}}}
  |map1={{{{Map|winner={game_info["result_formated"]}|white=1|eco={game_info["eco"]}|length={game_info["moves"]}}}}}
}}}}
  ''')

# Copypaste your final results by clicking a button

In [ ]:
def copy_to_clipboard(b):
    safe_text = output_text.replace('\n', '\\n').replace('"', '\\"')
    js_code = f'navigator.clipboard.writeText("{safe_text}");'
    display(Javascript(js_code))

copy_button = widgets.Button(description="Copy to Clipboard")
copy_button.on_click(copy_to_clipboard)

accordion = widgets.Accordion(children=[widgets.Textarea(value=output_text, layout={'width': '100%', 'height': '300px'})])
accordion.set_title(0, 'Hide/Show')

display(copy_button)
display(accordion)

# Participant Table generator
The flag is taken from the WhiteTeam and BlackTeam parameters. Therefore, be careful and check the output data.

In [ ]:
participants = {}

for game in games:
    if game['white_formated'] not in participants:
        participants[game['white_formated']] = game['white_team']
    if game['black_formated'] not in participants:
        participants[game['black_formated']] = game['black_team']

participants_output = []
for participant_name, participant_team in participants.items():
        participants_output.append(f"{{{{1Opponent|{participant_name}|flag={participant_team}|team=}}}}")
participants_joined = "\n".join(participants_output)
participants_formated = f'''==Participants==
{{{{ParticipantTable|count=1|colspan=|entrywidth=|title=|onlyNotable=
{participants_joined}
}}}}'''

participants_accordion = widgets.Accordion(children=[widgets.Textarea(value=participants_formated, layout={'width': '100%', 'height': '300px'})])
participants_accordion.set_title(0, 'Hide/Show')
display(participants_accordion)